In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# https://github.com/ome/openmicroscopy/blob/develop/examples/Training/python/Json_Api/Login.py
# https://docs.openmicroscopy.org/omero/5.6.1/developers/json-api.html

import requests
import getpass

USERNAME = "USER"
OMERO_WEB_HOST = "https://demo.openmicroscopy.org"
SERVER_NAME = "omero"
SESSION = requests.Session()
PASSWORD = getpass.getpass()

In [ ]:
# Start by getting supported versions from the base url...
api_url = '{0}/api/'.format(OMERO_WEB_HOST)
print("Starting at:", api_url)
r = SESSION.get(api_url)
# we get a list of versions
versions = r.json()['data']

# use most recent version...
version = versions[-1]
# get the 'base' url
base_url = version['url:base']
r = SESSION.get(base_url)
# which lists a bunch of urls as starting points
urls = r.json()
print(urls)
servers_url = urls['url:servers']
login_url = urls['url:login']
projects_url = urls['url:projects']
datasets_url = urls['url:datasets']
images_url = urls['url:images']
save_url = urls['url:save']
schema_url = urls['url:schema']

In [ ]:
# To login we need to get CSRF token
token_url = urls['url:token']
token = SESSION.get(token_url).json()['data']
print('CSRF token', token)
# We add this to our session header
# Needed for all POST, PUT, DELETE requests
SESSION.headers.update({'X-CSRFToken': token,
                        'Referer': login_url})

In [ ]:
# List the servers available to connect to
servers = SESSION.get(servers_url).json()['data']
print('Servers:')
for s in servers:
    print('-id:', s['id'])
    print(' name:', s['server'])
    print(' host:', s['host'])
    print(' port:', s['port'])
# find one called SERVER_NAME
servers = [s for s in servers if s['server'] == SERVER_NAME]
if len(servers) < 1:
    raise Exception("Found no server called '%s'" % SERVER_NAME)
server = servers[0]

# Login with username, password and token
payload = {'username': USERNAME,
           'password': PASSWORD,
           # 'csrfmiddlewaretoken': token,  # Using CSRFToken in header instead
           'server': server['id']}

r = SESSION.post(login_url, data=payload)
login_rsp = r.json()
assert r.status_code == 200
assert login_rsp['success']
eventContext = login_rsp['eventContext']
print('eventContext', eventContext)
# Can get our 'default' group
groupId = eventContext['groupId']

In [ ]:
# With successful login, request.session will contain
# OMERO session details and reconnect to OMERO on
# each subsequent call...

# List projects:
# Limit number of projects per page
payload = {'limit': 2}
data = SESSION.get(projects_url, params=payload).json()
assert len(data['data']) < 3
print("Projects:")
for p in data['data']:
    print('  ', p['@id'], p['Name'])

In [ ]:
# List datasets:
# Limit number of datasets per page
payload = {'limit': 2}
data = SESSION.get(datasets_url, params=payload).json()
assert len(data['data']) < 3
print("Datasets:")
for d in data['data']:
    print('  ', d['@id'], d['Name'])

In [ ]:
# Get images in datasets
payload = {'limit': 2}
data = SESSION.get(datasets_url, params=payload).json()
assert len(data['data']) < 3

for d in data['data']:
    dataset_images_url = '{0}{1}/images/'.format(datasets_url, d['@id'])
    
    data = SESSION.get(dataset_images_url, params=payload).json()
    assert len(data['data']) < 3
    print("Images:")
    
    for i in data['data']:
        print('  ', i['@id'], i['Name'])
        
        # Get image details
        image_details_url = '{0}{1}/'.format(images_url, i['@id'])
        data = SESSION.get(image_details_url, params=payload).json()

        print('  ', "Image details:")
        
        sizeX = data['data']['Pixels']['SizeX']
        sizeY = data['data']['Pixels']['SizeY']
        sizeZ = data['data']['Pixels']['SizeZ']
        sizeC = data['data']['Pixels']['SizeC']
        sizeT = data['data']['Pixels']['SizeT']
        
        print('  ','  ',
              "SizeX: {0}, SizeY: {1}, SizeZ: {2}, SizeC: {3}, SizeT: {4}"
              .format(sizeX, sizeY, sizeZ, sizeC, sizeT))
        channels = data['data']['Pixels']['Channels']
        print('  ','  ',"Number of channels: {0}".format(len(channels)))